<a href="https://colab.research.google.com/github/skaurl/P4-Model-Optimization/blob/main/Mask_Classfier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tflite-model-maker

     |████████████████████████████████| 593kB 7.6MB/s 
     |████████████████████████████████| 122kB 50.9MB/s 
     |████████████████████████████████| 849kB 37.0MB/s 
     |████████████████████████████████| 1.1MB 55.9MB/s 
     |████████████████████████████████| 645kB 66.6MB/s 
     |████████████████████████████████| 686kB 66.9MB/s 
     |████████████████████████████████| 112kB 69.4MB/s 
     |████████████████████████████████| 92kB 11.1MB/s 
     |████████████████████████████████| 71kB 10.8MB/s 
     |████████████████████████████████| 6.3MB 51.5MB/s 
     |████████████████████████████████| 174kB 59.5MB/s 
     |████████████████████████████████| 1.2MB 63.7MB/s 
     |████████████████████████████████| 358kB 59.5MB/s 
     |████████████████████████████████| 102kB 12.4MB/s 
     |████████████████████████████████| 38.2MB 134kB/s 
     |████████████████████████████████| 194kB 62.7MB/s 


In [2]:
import tarfile

f = tarfile.open('/content/drive/MyDrive/한양대학교/부스트캠프 AI Tech/[P4] 모델 최적화/train.tar.gz')
f.extractall()
f.close()

In [3]:
!find . -type f -name '._*' -delete

In [4]:
import os

import pandas as pd
import numpy as np

from tqdm import tqdm
from glob import glob
import shutil
import cv2
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt

In [5]:
os.makedirs('/content/train_dataset')
os.makedirs('/content/train_dataset/mask')
os.makedirs('/content/train_dataset/no_mask')
os.makedirs('/content/train_dataset/background')

os.makedirs('/content/val_dataset')
os.makedirs('/content/val_dataset/mask')
os.makedirs('/content/val_dataset/no_mask')
os.makedirs('/content/val_dataset/background')

In [6]:
img_info = pd.read_csv('/content/train/train.csv')

img_info_train, img_info_val = train_test_split(img_info, test_size=0.1, random_state=42)

In [7]:
for num in tqdm(range(len(img_info_train))):
    for name in os.listdir('/content/train/images/'+img_info_train.iloc[num,4]):
        original_path = '/content/train/images/'+img_info_train.iloc[num,4]+'/'+name
        destination_path = '/content/train_dataset/'

        if name[:4] == 'mask':
            img = cv2.imread(original_path)[256-96:256+96,192-96:192+96]
            img = cv2.resize(img,dsize=(224,224))
            cv2.imwrite(destination_path+'mask/'+img_info_train.iloc[num,4]+'_a_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,224-96:224+96]
            img = cv2.resize(img,dsize=(224,224))
            cv2.imwrite(destination_path+'mask/'+img_info_train.iloc[num,4]+'_b_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,160-96:160+96]
            img = cv2.resize(img,dsize=(224,224))
            cv2.imwrite(destination_path+'mask/'+img_info_train.iloc[num,4]+'_c_'+name,img)
        elif name[:4] == 'inco':
            img = cv2.imread(original_path)[256-96:256+96,192-96:192+96]
            img = cv2.resize(img,dsize=(224,224))
            cv2.imwrite(destination_path+'no_mask/'+img_info_train.iloc[num,4]+'_a_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,224-96:224+96]
            img = cv2.resize(img,dsize=(224,224))
            cv2.imwrite(destination_path+'no_mask/'+img_info_train.iloc[num,4]+'_b_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,160-96:160+96]
            img = cv2.resize(img,dsize=(224,224))
            cv2.imwrite(destination_path+'no_mask/'+img_info_train.iloc[num,4]+'_c_'+name,img)
        elif name[:4] == 'norm':
            img = cv2.imread(original_path)[256-96:256+96,192-96:192+96]
            img = cv2.resize(img,dsize=(224,224))
            cv2.imwrite(destination_path+'no_mask/'+img_info_train.iloc[num,4]+'_a_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,224-96:224+96]
            img = cv2.resize(img,dsize=(224,224))
            cv2.imwrite(destination_path+'no_mask/'+img_info_train.iloc[num,4]+'_b_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,160-96:160+96]
            img = cv2.resize(img,dsize=(224,224))
            cv2.imwrite(destination_path+'no_mask/'+img_info_train.iloc[num,4]+'_c_'+name,img)
            
            img = cv2.imread(original_path)[:128,:128]
            img = cv2.resize(img,dsize=(224,224))
            cv2.imwrite(destination_path+'background/'+img_info_train.iloc[num,4]+'_a_'+name,img)
            img = cv2.imread(original_path)[256-64:256+64,:128]
            img = cv2.resize(img,dsize=(224,224))
            cv2.imwrite(destination_path+'background/'+img_info_train.iloc[num,4]+'_b_'+name,img)
            img = cv2.imread(original_path)[-128:,:128]
            img = cv2.resize(img,dsize=(224,224))
            cv2.imwrite(destination_path+'background/'+img_info_train.iloc[num,4]+'_c_'+name,img)
            img = cv2.imread(original_path)[:128,192-64:192+64]
            img = cv2.resize(img,dsize=(224,224))
            cv2.imwrite(destination_path+'background/'+img_info_train.iloc[num,4]+'_d_'+name,img)
            img = cv2.imread(original_path)[-128:,192-64:192+64]
            img = cv2.resize(img,dsize=(224,224))
            cv2.imwrite(destination_path+'background/'+img_info_train.iloc[num,4]+'_e_'+name,img)
            img = cv2.imread(original_path)[:128,-128:]
            img = cv2.resize(img,dsize=(224,224))
            cv2.imwrite(destination_path+'background/'+img_info_train.iloc[num,4]+'_f_'+name,img)
            img = cv2.imread(original_path)[256-64:256+64,-128:]
            img = cv2.resize(img,dsize=(224,224))
            cv2.imwrite(destination_path+'background/'+img_info_train.iloc[num,4]+'_g_'+name,img)
            img = cv2.imread(original_path)[-128:,-128:]
            img = cv2.resize(img,dsize=(224,224))
            cv2.imwrite(destination_path+'background/'+img_info_train.iloc[num,4]+'_h_'+name,img)

100%|██████████| 2430/2430 [03:58<00:00, 10.17it/s]


In [8]:
for num in tqdm(range(len(img_info_val))):
    for name in os.listdir('/content/train/images/'+img_info_val.iloc[num,4]):
        original_path = '/content/train/images/'+img_info_val.iloc[num,4]+'/'+name
        destination_path = '/content/val_dataset/'

        if name[:4] == 'mask':
            img = cv2.imread(original_path)[256-96:256+96,192-96:192+96]
            img = cv2.resize(img,dsize=(224,224))
            cv2.imwrite(destination_path+'mask/'+img_info_val.iloc[num,4]+'_a_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,224-96:224+96]
            img = cv2.resize(img,dsize=(224,224))
            cv2.imwrite(destination_path+'mask/'+img_info_val.iloc[num,4]+'_b_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,160-96:160+96]
            img = cv2.resize(img,dsize=(224,224))
            cv2.imwrite(destination_path+'mask/'+img_info_val.iloc[num,4]+'_c_'+name,img)
        elif name[:4] == 'inco':
            img = cv2.imread(original_path)[256-96:256+96,192-96:192+96]
            img = cv2.resize(img,dsize=(224,224))
            cv2.imwrite(destination_path+'no_mask/'+img_info_val.iloc[num,4]+'_a_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,224-96:224+96]
            img = cv2.resize(img,dsize=(224,224))
            cv2.imwrite(destination_path+'no_mask/'+img_info_val.iloc[num,4]+'_b_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,160-96:160+96]
            img = cv2.resize(img,dsize=(224,224))
            cv2.imwrite(destination_path+'no_mask/'+img_info_val.iloc[num,4]+'_c_'+name,img)
        elif name[:4] == 'norm':
            img = cv2.imread(original_path)[256-96:256+96,192-96:192+96]
            img = cv2.resize(img,dsize=(224,224))
            cv2.imwrite(destination_path+'no_mask/'+img_info_val.iloc[num,4]+'_a_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,224-96:224+96]
            img = cv2.resize(img,dsize=(224,224))
            cv2.imwrite(destination_path+'no_mask/'+img_info_val.iloc[num,4]+'_b_'+name,img)
            img = cv2.imread(original_path)[256-96:256+96,160-96:160+96]
            img = cv2.resize(img,dsize=(224,224))
            cv2.imwrite(destination_path+'no_mask/'+img_info_val.iloc[num,4]+'_c_'+name,img)
            
            img = cv2.imread(original_path)[:128,:128]
            img = cv2.resize(img,dsize=(224,224))
            cv2.imwrite(destination_path+'background/'+img_info_val.iloc[num,4]+'_a_'+name,img)
            img = cv2.imread(original_path)[256-64:256+64,:128]
            img = cv2.resize(img,dsize=(224,224))
            cv2.imwrite(destination_path+'background/'+img_info_val.iloc[num,4]+'_b_'+name,img)
            img = cv2.imread(original_path)[-128:,:128]
            img = cv2.resize(img,dsize=(224,224))
            cv2.imwrite(destination_path+'background/'+img_info_val.iloc[num,4]+'_c_'+name,img)
            img = cv2.imread(original_path)[:128,192-64:192+64]
            img = cv2.resize(img,dsize=(224,224))
            cv2.imwrite(destination_path+'background/'+img_info_val.iloc[num,4]+'_d_'+name,img)
            img = cv2.imread(original_path)[-128:,192-64:192+64]
            img = cv2.resize(img,dsize=(224,224))
            cv2.imwrite(destination_path+'background/'+img_info_val.iloc[num,4]+'_e_'+name,img)
            img = cv2.imread(original_path)[:128,-128:]
            img = cv2.resize(img,dsize=(224,224))
            cv2.imwrite(destination_path+'background/'+img_info_val.iloc[num,4]+'_f_'+name,img)
            img = cv2.imread(original_path)[256-64:256+64,-128:]
            img = cv2.resize(img,dsize=(224,224))
            cv2.imwrite(destination_path+'background/'+img_info_val.iloc[num,4]+'_g_'+name,img)
            img = cv2.imread(original_path)[-128:,-128:]
            img = cv2.resize(img,dsize=(224,224))
            cv2.imwrite(destination_path+'background/'+img_info_val.iloc[num,4]+'_h_'+name,img)

100%|██████████| 270/270 [00:26<00:00, 10.27it/s]


In [9]:
train_data = DataLoader.from_folder('/content/train_dataset')
val_data = DataLoader.from_folder('/content/val_dataset')

INFO:tensorflow:Load image with size: 70470, num_label: 3, labels: background, mask, no_mask.
INFO:tensorflow:Load image with size: 7830, num_label: 3, labels: background, mask, no_mask.


In [10]:
custom = image_classifier.ModelSpec(uri='https://tfhub.dev/google/imagenet/mobilenet_v1_025_224/classification/5') # Total params: 478,807 | {'accuracy': 0.969220945083014}
# custom = image_classifier.ModelSpec(uri='https://tfhub.dev/google/imagenet/mobilenet_v3_small_075_224/classification/5')
# custom = image_classifier.ModelSpec(uri='https://tfhub.dev/tensorflow/efficientnet/lite0/classification/2')
# custom = image_classifier.ModelSpec(uri='https://tfhub.dev/tensorflow/efficientnet/b0/classification/1')

In [11]:
model = image_classifier.create(train_data=train_data,
                                model_spec=custom,
                                validation_data=val_data,
                                batch_size=512,
                                epochs=50,
                                learning_rate=5e-4,
                                shuffle=True
                                )

INFO:tensorflow:Retraining the models...


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2 (HubKer (None, 1001)              475801    
_________________________________________________________________
dropout (Dropout)            (None, 1001)              0         
_________________________________________________________________
dense (Dense)                (None, 3)                 3006      
Total params: 478,807
Trainable params: 3,006
Non-trainable params: 475,801
_________________________________________________________________
None
Epoch 1/50


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


137/137 [==============================] - 100s 600ms/step - loss: 0.9957 - accuracy: 0.8270 - val_loss: 0.5159 - val_accuracy: 0.9557
Epoch 2/50
137/137 [==============================] - 81s 597ms/step - loss: 0.6505 - accuracy: 0.8944 - val_loss: 0.4556 - val_accuracy: 0.9669
Epoch 3/50
137/137 [==============================] - 82s 602ms/step - loss: 0.5750 - accuracy: 0.9115 - val_loss: 0.4264 - val_accuracy: 0.9708
Epoch 4/50
137/137 [==============================] - 82s 599ms/step - loss: 0.5277 - accuracy: 0.9214 - val_loss: 0.4081 - val_accuracy: 0.9691
Epoch 5/50
137/137 [==============================] - 80s 589ms/step - loss: 0.4936 - accuracy: 0.9299 - val_loss: 0.3984 - val_accuracy: 0.9745
Epoch 6/50
137/137 [==============================] - 80s 584ms/step - loss: 0.4704 - accuracy: 0.9384 - val_loss: 0.3921 - val_accuracy: 0.9730
Epoch 7/50
137/137 [==============================] - 80s 584ms/step - loss: 0.4522 - accuracy: 0.9446 - val_loss: 0.3830 - val_accuracy: 0.

In [12]:
config = QuantizationConfig.for_dynamic()

In [13]:
model.export(export_dir='/content/drive/MyDrive/한양대학교/부스트캠프 AI Tech/[P4] 모델 최적화/', quantization_config=config)

INFO:tensorflow:Assets written to: /tmp/tmpbx5u0b8v/assets


INFO:tensorflow:Assets written to: /tmp/tmpbx5u0b8v/assets


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Saving labels in /tmp/tmpe87cf1mz/labels.txt


INFO:tensorflow:Saving labels in /tmp/tmpe87cf1mz/labels.txt


INFO:tensorflow:TensorFlow Lite model exported successfully: /content/drive/MyDrive/한양대학교/부스트캠프 AI Tech/[P4] 모델 최적화/model.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: /content/drive/MyDrive/한양대학교/부스트캠프 AI Tech/[P4] 모델 최적화/model.tflite


In [14]:
model.evaluate_tflite('/content/drive/MyDrive/한양대학교/부스트캠프 AI Tech/[P4] 모델 최적화/model.tflite', val_data)

{'accuracy': 0.969220945083014}